# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Load, Engineer & Connect</span>

<span style="font-width:bold; font-size: 1.4rem;"> This is the first part of the quick start series of tutorials about Hopsworks Feature Store. As part of this first module, we will work with data related to credit card transactions. 
The objective of this tutorial is to demonstrate how to work with the **Hopworks Feature Store**  for batch data with a goal of training and deploying a model that can predict fraudulent transactions.</span>

## **🗒️ This notebook is divided in 3 sections:** 
1. Loading the data and feature engineeing,
2. Connect to the Hopsworks feature store,
3. Create feature groups and upload them to the feature store.

![tutorial-flow](images/01_featuregroups.png)

First of all we will load the data and do some feature engineering on it.

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

The data we will use comes from three different CSV files:

- `credit_cards.csv`: credit card information such as expiration date and provider.
- `transactions.csv`: transaction information such as timestamp, location, and the amount. Importantly, the binary `fraud_label` variable tells us whether a transaction was fraudulent or not.
- `profiles.csv`: credit card user information such as birthdate and city of residence.

We can conceptualize these CSV files as originating from separate data sources.
**All three files have a credit card number column `cc_num` in common, which we can use for joins.**

Let's go ahead and load the data.

In [35]:
import pandas as pd
credit_cards_df = pd.read_csv("https://repo.hops.works/dev/davit/card_fraud_online/credit_cards.csv")
credit_cards_df.head(3)

,cc_num,provider,expires
0,4796807885357879,visa,05/23
1,4529266636192966,visa,03/22
2,4922690008243953,visa,02/27


In [36]:
profiles_df = pd.read_csv("https://repo.hops.works/dev/davit/card_fraud_online/profiles.csv", parse_dates=["birthdate"])
profiles_df.head(3)

,name,sex,mail,birthdate,City,Country,cc_num
0,Tonya Gregory,F,sandratorres@hotmail.com,1976-01-16,Far Rockaway,US,4796807885357879
1,Lisa Gilbert,F,michael53@yahoo.com,1986-09-30,Encinitas,US,4529266636192966
2,Carolyn Meyer,F,anthony47@yahoo.com,2001-07-13,Canton,US,4922690008243953


In [37]:
trans_df = pd.read_csv("https://repo.hops.works/dev/davit/card_fraud_online/transactions.csv", parse_dates=["datetime"])
trans_df.head(3)

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,11df919988c134d97bbff2678eb68e22,2022-01-01 00:00:24,4473593503484549,Health/Beauty,62.95,42.30865,-83.48216,Canton,US,0
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,2022-01-01 00:00:56,4272465718946864,Grocery,85.45,33.52253,-117.70755,Laguna Niguel,US,0
2,e627f5d9a9739833bd52d2da51761fc3,2022-01-01 00:02:32,4104216579248948,Domestic Transport,21.63,37.60876,-77.37331,Mechanicsville,US,0


In [38]:
trans_df = trans_df[trans_df.category == "Cash Withdrawal"].reset_index(level=0, drop=True)
trans_df["country"] = trans_df["country"].fillna("US")
profiles_df = profiles_df[profiles_df.cc_num.isin(trans_df.cc_num.unique())].reset_index(level=0, drop=True)
credit_cards_df = credit_cards_df[credit_cards_df.cc_num.isin(trans_df.cc_num.unique())].reset_index(level=0, drop=True)

In [39]:
trans_df

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,ada19b194d8b93d502f16309c7275e36,2022-02-20 05:14:58,4148855537936983,Cash Withdrawal,2845.89,27.107439,-82.462260,Venice,US,0
1,69b38c908b93b58b78250f4d66e8a9ec,2022-02-20 17:14:58,4148855537936983,Cash Withdrawal,268.28,27.105581,-82.454755,Venice,US,0
2,7406892d42247a0f849098c711787144,2022-02-21 05:14:58,4148855537936983,Cash Withdrawal,36.63,27.102719,-82.448723,Venice,US,0
3,cf55a58e38c93fbfba37f2f636b63dfc,2022-01-16 17:03:04,4148855537936983,Cash Withdrawal,176.23,38.961261,-76.986751,Chillum,US,0
4,ee28f7df1237e124a06537eb4fdfc89d,2022-01-17 21:03:04,4148855537936983,Cash Withdrawal,27.61,38.952633,-76.989699,Chillum,US,0
...,...,...,...,...,...,...,...,...,...,...
365115,50c55d97716ab5f9c435d8bd46a717a7,2022-03-09 16:39:23,4904072446387894,Cash Withdrawal,411.90,30.158790,-96.412289,Brenham,US,0
365116,1c60cb8676fe60151a0a9168dceaf80e,2022-03-11 17:39:23,4904072446387894,Cash Withdrawal,610.58,30.167319,-96.408537,Brenham,US,0
365117,9582563f06e0e46a6b6e8bd961360f0b,2022-03-13 18:39:23,4904072446387894,Cash Withdrawal,9246.22,30.170026,-96.402064,Brenham,US,0
365118,bc380ed57c86a3f2e72f845bcd192388,2022-03-15 19:39:23,4904072446387894,Cash Withdrawal,252.55,30.178488,-96.403905,Brenham,US,0


## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning we will create additional features based on these patterns. In particular, we will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

In [40]:
import numpy as np

In [41]:
# Compute age at transaction.
age_df = trans_df.merge(profiles_df, on="cc_num", how="left")
trans_df["age_at_transaction"] = (age_df["datetime"] - age_df["birthdate"]) / np.timedelta64(1, "Y")

# Compute days until card expires.
card_expiry_df = trans_df.merge(credit_cards_df, on="cc_num", how="left")
card_expiry_df["expires"] = pd.to_datetime(card_expiry_df["expires"], format="%m/%y")
trans_df["days_until_card_expires"] = (card_expiry_df["expires"] - card_expiry_df["datetime"]) / np.timedelta64(1, "D")

trans_df[["age_at_transaction", "days_until_card_expires"]].head()

,age_at_transaction,days_until_card_expires
0,57.691037,434.781273
1,57.692406,434.281273
2,57.693775,433.781273
3,57.596557,469.289537
4,57.599751,468.122870


Next, we create features that for each credit card aggregate data from multiple time steps.

We start by computing the distance between consecutive transactions, which we will call `loc_delta`.
Here we use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

In [42]:
from math import radians

# Do some simple preprocessing.
trans_df.sort_values("datetime", inplace=True)
trans_df[["longitude", "latitude"]] = trans_df[["longitude", "latitude"]].applymap(radians)

def haversine(long, lat):
    """Compute Haversine distance between each consecutive coordinate in (long, lat)."""

    long_shifted = long.shift()
    lat_shifted = lat.shift()
    long_diff = long_shifted - long
    lat_diff = lat_shifted - lat

    a = np.sin(lat_diff/2.0)**2
    b = np.cos(lat) * np.cos(lat_shifted) * np.sin(long_diff/2.0)**2
    c = 2*np.arcsin(np.sqrt(a + b))

    return c


trans_df["loc_delta"] = trans_df.groupby("cc_num")\
    .apply(lambda x : haversine(x["longitude"], x["latitude"]))\
    .reset_index(level=0, drop=True)\
    .fillna(0)

Next we compute windowed aggregates. Here we will use 4-hour windows, but feel free to experiment with different window lengths by setting `window_len` below to a value of your choice.

In [43]:
window_len = "4h"
# Moving average of location difference between consecutive transactions.
cc_group = trans_df[["cc_num", "loc_delta", "datetime"]].groupby("cc_num").rolling(window_len, on="datetime").mean()
df_4h_loc_delta_mavg = pd.DataFrame(cc_group)
df_4h_loc_delta_mavg.columns = ["loc_delta_mavg", "datetime"]
df_4h_loc_delta_mavg = df_4h_loc_delta_mavg.reset_index(level=["cc_num"])
df_4h_loc_delta_mavg = df_4h_loc_delta_mavg.drop(columns=["cc_num", "datetime"])
df_4h_loc_delta_mavg = df_4h_loc_delta_mavg.sort_index()
window_aggs_df = df_4h_loc_delta_mavg

window_aggs_df = window_aggs_df.merge(trans_df[["cc_num", "datetime"]].sort_index(),left_index=True, right_index=True)
window_aggs_df.tail()

,loc_delta_mavg,cc_num,datetime
365115,0.361546,4904072446387894,2022-03-09 16:39:23
365116,0.469146,4904072446387894,2022-03-11 17:39:23
365117,0.000140,4904072446387894,2022-03-13 18:39:23
365118,0.144911,4904072446387894,2022-03-15 19:39:23
365119,0.443975,4904072446387894,2022-03-17 20:39:23


### Convert date time object to unix epoch in milliseconds

In [44]:
trans_df.datetime = trans_df.datetime.values.astype(np.int64) // 10 ** 6
window_aggs_df.datetime = window_aggs_df.datetime.values.astype(np.int64) // 10 ** 6

## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features. In our case, we will create a feature group for the transaction data and a feature group for the windowed aggregations on the transaction data. Both will have `tid` as primary key, which will allow us to join them when creating a dataset in the next tutorial.

Feature groups can also be used to define a namespace for features. For instance, in a real-life setting we would likely want to experiment with different window lengths. In that case, we can create feature groups with identical schema for each window length. 

Before we can create a feature group we need to connect to our feature store.

In [45]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


To create a feature group we need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [46]:
trans_fg = fs.create_feature_group(
    name="transactions",
    version="1",
    description="Transaction data",
    primary_key=['cc_num'],
    event_time=['datetime'],
    online_enabled=True
)


Here we have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

At this point, we have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent we populate it with its associated data using the `save` function.

In [48]:
trans_fg.save(trans_df)

Feature Group created successfully, explore it at https://73bbb040-ed8b-11ec-a289-2979fdcaf1e8.cloud.hopsworks.ai/p/120/fs/68/fg/23
Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://73bbb040-ed8b-11ec-a289-2979fdcaf1e8.cloud.hopsworks.ai/p/120/jobs/named/transactions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fc1dff4b910>, None)

We can move on and do the same thing for the feature group with our windows aggregation.

In [49]:
window_aggs_fg = fs.create_feature_group(
    name=f"transactions_{window_len}_aggs",
    description=f"Aggregate transaction data over {window_len} windows.",
    primary_key=['cc_num'],
    event_time=['datetime'],
    online_enabled=True    
)

In [50]:
window_aggs_fg.save(window_aggs_df)

Feature Group created successfully, explore it at https://73bbb040-ed8b-11ec-a289-2979fdcaf1e8.cloud.hopsworks.ai/p/120/fs/68/fg/24
Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://73bbb040-ed8b-11ec-a289-2979fdcaf1e8.cloud.hopsworks.ai/p/120/jobs/named/transactions_4h_aggs_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fc1fa3d0850>, None)

Both feature groups are now accessible and searchable in the UI

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02 </span>

In the following notebook we will use our feature groups to create a dataset we can train a model on.